# Legal Robots: Text Mining

Nun hat unser **A.A.** schon eine ganze Menge Daten gesammelt und ist in der Lage, 
mit wenigen Code-Zeilen Statistiken zu den Metadaten deutscher Gerichtsentscheidungen zu generieren. 
Mit dem Inhalt dieser Entscheidungen hat er sich allerdings bisher wenig befasst. 

Seitdem er die Bibliotheksvorgänge digitalisiert hat, wird **A.A.** außerdem von einem der Bibliotheksroboter, genannt **Krit.**, auf Schritt und Schritt verfolgt. 
**Krit.** schlägt eine mechanische Inspektion der gesammelten Dokumente vor - *ad fontes*, so ist ihm aus seiner Zeit in der Abteilung *Rechtsgeschichte* noch gut in Erinnerung. 
Doch **A.A.** weiß, dass er nicht die Zeit hat, um alle Entscheidungen zu lesen, und die kursorische Lektüre einzelner Entscheidungen kommt für ihn mangels intersubjektiver Überprüfbarkeit von vornherein nicht in Betracht. 

**A.A.** entscheidet sich daher, 
aus den Entscheidungstexten weitere Daten zu generieren, mit denen er den Inhalt der gesammelten Entscheidungen zu charakterisieren vermag. 

____

## Aufgabe 1: Grundlagen

a) Anhand welcher Merkmale könnte man Gerichtsentscheidungen auf Basis ihres Textes charakterisieren?
Wie ließen sich diese Merkmale algorithmisch operationalisieren?

b) Was versteht man unter NLP? 
Wozu dienen Sentence Splitting, Tokenization, Parsing, POS-Tagging, Lemmatization und NER? 

c) Was versteht man unter Regex?
Inwieweit könnten Regex **A.A.** möglicherweise behilflich sein?
Welche Vor- und Nachteile besitzen sie (auf den ersten Blick) im Vergleich zu einer elaborierteren NLP-Pipeline?
___

**A.A.** entschließt sich, angesichts der Kürze der Zeit zunächst auf *Regex* zurückzugreifen. 

Wir wollen ihn in seinem Aufbegehren gegen **h.M.** unterstützen, indem wir ebenfalls Regex-basierte Analysen durchführen. 
Dazu verbinden wir uns zunächst wieder mit unserer Datenbank:

In [1]:
import sqlite3
%load_ext sql
%sql sqlite:///rii.db
con = sqlite3.connect('rii.db')

Wie sah noch einmal das Schema aus?

In [54]:
%sql select * from entscheidungen limit 0

Done.


doknr,ecli,gertyp,gerort,spruchkoerper,entscheidungsdatum,aktenzeichen,doktyp,norm,vorinstanz,identifier,volltext


Wir interessieren uns hier vor allem für den Inhalt der letzten Spalte: `volltext`.

Um unseren Namensraum (*namespace*) nicht unnötig zu verunreinigen, schreiben wir für unsere Analyse schnell einige Helferfunktionen:

In [3]:
def get_volltext(doknr:str):
    """TODO: Kommentar"""
    cursor = con.cursor()
    cursor.execute("SELECT volltext FROM entscheidungen WHERE doknr = ?", (doknr,))
    return cursor.fetchone()[0]

In [33]:
import re
def get_matches(regex:str, con):
    """TODO: Kommentar"""
    cursor = con.cursor()
    matches = {}
    for row in cursor.execute("SELECT doknr, volltext FROM entscheidungen"):
        doknr = row[0]
        text = row[1]
        dokmatches = re.findall(regex, text)
        if dokmatches:
            matches[doknr] = dokmatches
    return matches

In [64]:
def get_number_of_matches(matches, sort=False, reverse=True):
    """TODO: Kommentar"""
    match_numbers = {}
    for key in matches.keys():
        match_numbers[key] = len(matches[key])
    return match_numbers if not sort else sorted(
        [(k, match_numbers[k]) for k in match_numbers.keys()], 
        key=lambda tup:tup[1], reverse=reverse)

Nun können wir beispielsweise untersuchen, in welchem Kontext die Abkürzung `GmbH` in unserem Entscheidungssatz vorkommt,...

In [90]:
pattern = '.{0,20}\WGmbH\W.{0,20}'
matches = get_matches(pattern, con)
matches

{'JURE100055033': ['die Forderung der U.\xa0GmbH gegen die Schuldneri',
  'llmächtigten" der U.\xa0GmbH betreffend eine ange'],
 'JURE100057309': ['eschäftsführer einer GmbH einstimmig, eine geb'],
 'JURE100057819': ['sverhältnisse der B.\xa0GmbH B.\xa0(im folgenden: BI'],
 'JURE100057867': [' das Vermögen der H. GmbH (künftig: Gemeinschu'],
 'JURE100059619': ['vollstreckung der N. GmbH wegen einer Forderun'],
 'JURE100060891': ['nternational Service GmbH" (Beklagte zu 1) und',
  ' "Bundesdruckerei GmbH" (Beklagte zu 2) zu '],
 'JURE100061265': ['Geschäftsführers der GmbH\n',
  ' Geschäftsführer der GmbH nach den geltenden S'],
 'JURE100061287': [' Juni 1996 die E.\xa0E.\xa0GmbH (fortan E.), deren G'],
 'JURE100061737': ['sten einer Filmfonds-GmbH & Co. KG: Umfang der',
  'bH (im Folgenden: IT GmbH) unterrichtet worden',
  'e dahin, dass die IT GmbH - neben der prospekt',
  'eiligung befasste IT GmbH auch noch diese Verg',
  'Umstand, dass die IT GmbH in der Person ihres ',
  'derbehan

...und wir können ermitteln, in welchen Entscheidungen `GmbH` besonders häufig vorkommt...

In [91]:
match_numbers = get_number_of_matches(matches, True)
match_numbers[:5]

[('JURE109006785', 61),
 ('JURE109005552', 60),
 ('JURE140020450', 60),
 ('JURE150011999', 53),
 ('JURE140008031', 51)]

____
## Aufgabe 2: Mit Regex arbeiten
a) Stelle sicher, dass du verstanden hast, was unsere Helfer-Funktionen tun, 
und schreibe an den mit `TODO` gekennzeichneten Stellen entsprechende `Docstrings`. 
Welche Funktion(en) müsstest du wo ändern, wenn du nur Entscheidungen eines bestimmten Gerichtes durchsuchen wolltest?
___
b) Mache dich mit der Syntax von [Python Regex](https://docs.python.org/3/library/re.html) vertraut und finde heraus, was die einzelnen Elemente in unserem GmbH-`pattern` bewirken.
___
c) Allein mit der `doknr` und der Anzahl der Treffer können wir wenig anfangen. 
* Schreibe eine Helfer-Funktion, die zu einer gegebenen `doknr` den Datensatz aus `entscheidungen` ausliest. 
* Schreibe eine Funktion, die für jede `doknr` die Anzahl der Treffer für ein bestimmtes Regex-Muster in eine sinnvoll benannte zusätzliche Spalte schreibt.
___
d) Beantworte mithilfe von Python Regex folgende Fragen und persistiere die Antworten mithilfe der zweiten Funktion aus c) in deiner Datenbank:
* Wie häufig wurde in den Entscheidungen, die in deinem Datensatz enthalten sind, der Palandt zitiert, wie häufig der Staudinger? 
* In welchen Entscheidungen geht es um § 242 BGB, in welchen um § 242 StGB?
* Wie häufig wird in den Entscheidungen auf die herrschende Meinung oder herrschende Lehre Bezug genommen, wie häufig auf kritische Stimmen oder andere Ansichten?
* Wie häufig fließen Erwägungen in eine Entscheidung mit ein, die vom Spruchkörper als *quantitativ* eingeordnet werden?
* In welchen Entscheidungen hat man sich wie intensiv mit Robotern befasst?

> Welche Probleme stellen sich in den Unteraufgaben von Teilaufgabe d) hinsichtlich der Genauigkeit deiner Untersuchungen?

___
e) Finde mithilfe von Anfragen an deine aktualisierte Datenbank heraus, 
wie deine Treffer über deine Entscheidungen und deren Metadaten-Merkmale verteilt sind. 
Gehe dabei jedenfalls folgenden Fragen nach:
* In welchen Entscheidungen wird sowohl der Palandt zitiert als auch § 242 BGB erwähnt? 
Wie sieht es mit dem Staudinger und § 242 BGB aus? Gibt es Entscheidungen, die den Palandt oder den Staudinger im Kontext von § 242 StGB erwähnen?
* Welche Gerichte (`gertyp`) haben sich häufig, welche weniger häufig mit Robotern auseinandergesetzt; wie ist dieselbe Frage für quantitative Erwägungen zu beantworten? 
In welchen Jahren haben diese Materien die Gerichte besonders beschäftigt?
* Von welchen Senaten (`spruchkoerper`) des BGH wird der Palandt besonders intensiv zitiert, von welchen der Staudinger? Welche BGH-Senate geben anderen Ansichten, welche herrschenden Ansichten besonders viel Raum?

> Welche Schwierigkeiten treten auf, wenn du versuchst, diese in natürlicher Sprache formulierten Aufgaben in Interaktion mit dem Computer zu beantworten?

____

## Aufgabe 3: NLP-Universum erkunden

a) Welche NLP-Bibliotheken stehen in Python zur Verfügung, die über Regex hinausgehende Funktionalitäten bereitstellen?
Welche dieser Bibliotheken könnten für die Verarbeitung deutschsprachiger Rechtstexte geeignet sein? 

b) Bei welchen NLP-Aktivitäten werden in jedem Fall Anpassungen vorzunehmen sein und warum?

b) Wähle aus den für deutschsprachige Texte geeigneten Bibliotheken eine aus und bringe sie auf deinem Rechner mit der für Deutsch erforderlichen Konfiguration zum Laufen. 
Lies aus deinem `rii`-Ordner ein paar Entscheidungen ein und probiere einige der standardmäßig zur Verfügung stehenden Methoden aus. 
Welche NLP-Arbeitsschritte sind für die Untersuchung von Rechtstexten besonders wichtig, welche besonders schwierig?